In [ ]:
from optimization import *
import h5py

In [121]:
# Todo: replace this function with actual file reading
def readHamiltonian_params(file, idx, N):
    Jijalphabeta = np.random.rand(3, 3, N, N)
    h = np.random.rand(3, N)

    # Delete 90% of the elements randomly in Jijalphabeta
    mask_J = np.random.rand(3, 3, N, N) < 0.1
    Jijalphabeta = Jijalphabeta * mask_J

    # Delete 90% of the elements randomly in h
    mask_h = np.random.rand(3, N) < 0.1
    h = h * mask_h

    return Jijalphabeta, h


def generate_tfim_params(N, J=1.0, h=1.0):
    """
    Generates Jij and h arrays for the transverse field Ising model (TFIM):
    H = -J sum_{<i,j>} sigma^z_i sigma^z_j - h sum_i sigma^x_i

    Args:
        N (int): Number of spins (1D chain with periodic boundary).
        J (float): Coupling strength.
        h (float): Transverse field strength.

    Returns:
        Jij (np.ndarray): shape (3, 3, N, N), only Jij[2,2,i,j] nonzero for nearest neighbors.
        h (np.ndarray): shape (3, N), only h[0,i] nonzero (x direction).
    """
    Jij = np.zeros((3, 3, N, N))
    for i in range(N):
        j = (i + 1) % N  # periodic boundary
        Jij[2, 2, i, j] = -J
        Jij[2, 2, j, i] = -J  # symmetric

    h_arr = np.zeros((3, N))
    h_arr[0, :] = -h  # transverse field in x direction

    return Jij, h_arr

In [125]:
# Jij, h = readHamiltonian_params("hamiltonian_params.txt", 0, 8)
Jij, h = generate_tfim_params(12, J=-1.0, h=0.8)
H = construct_hamiltonian(Jij, h)
exact_ground_energy,exact_ground_state = nk.exact.lanczos_ed(H, k = 1, compute_eigenvectors = True)
print("Exact ground state energy:", exact_ground_energy)

Exact ground state energy: [-14.02114449]


In [126]:

params = {
    "alpha": 5,
    "seed": 1234,
    "learning_rate": 0.01,
    "n_iter": 500,
    "show_progress": True,
}
out = optimize_rbm(H, params)


100%|██████████| 500/500 [00:07<00:00, 68.20it/s, Energy=-1.402e+01 ± 0.000e+00 [σ²=8.518e-03]]


In [136]:
out["log"]

()

In [135]:
def write_output(H, out):
    vstate = out["vstate"]
    psi = vstate.to_array()
    # Try to get the last energy value if available, else set to None
    en_var = None
    if isinstance(out.get("log", None), dict) and "Energy" in out["log"]:
        en_var = out["log"]["Energy"][-1]
    exact_ground_energy, psi_0 = nk.exact.lanczos_ed(H, k=1, compute_eigenvectors=True)
    psi_0 = psi_0[:, 0]  # Get the ground state vector
    with h5py.File("output.h5", "w") as f:
        f.create_dataset("psi", data=psi)
        f.create_dataset("en_var", data=en_var if en_var is not None else -1)
        f.create_dataset("exact_ground_energy", data=exact_ground_energy)
        f.create_dataset("psi_0", data=psi_0)

write_output(H, out)


In [129]:
np.linalg.norm(out["vstate"].to_array() - exact_ground_state[:,0])

np.float64(0.01384851176754883)